In [1]:
# The following python is used to train discrimiator and generator and
# save the compound model to use later to generate handwritten digits

In [1]:
# importing all the required dependencies
from numpy import expand_dims, zeros, ones, vstack
from numpy.random import randn, randint
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# The discriminator model to distinguish between real and fake digits
#  of the MNIST dataset
def discriminator(in_shape = (28, 28, 1)):
    disc = Sequential()
    disc.add(Conv2D(64, (3, 3), strides = (2, 2), padding = 'same', input_shape = in_shape))
    disc.add(LeakyReLU(alpha = 0.2))
    disc.add(Dropout(0.4))
    disc.add(Conv2D(64, (3, 3), strides = (2, 2), padding = 'same'))
    disc.add(LeakyReLU(alpha = 0.2))
    disc.add(Dropout(0.4))
    disc.add(Flatten())
    disc.add(Dense(1, activation = 'sigmoid'))
    o = Adam(lr = 0.0002, beta_1 = 0.5)
    disc.compile(loss = 'binary_crossentropy', optimizer = o, metrics = ['accuracy'])
    return disc

In [3]:
# The generator model to generate handwritten digits
def generator(latent_dim):
    gen = Sequential()
    nodes = 128 * 7 * 7
    gen.add(Dense(nodes, input_dim = latent_dim))
    gen.add(LeakyReLU(alpha = 0.2))
    gen.add(Reshape((7, 7, 128)))
    # to unsample to 14x14 images
    gen.add(Conv2DTranspose(128, (4, 4), strides = (2, 2), padding = 'same'))
    gen.add(LeakyReLU(alpha = 0.2))
    # unsampling t 28x28 images
    gen.add(Conv2DTranspose(128, (4, 4), strides = (2, 2), padding = 'same'))
    gen.add(LeakyReLU(alpha = 0.2))
    gen.add(Conv2D(1, (7, 7), activation = 'sigmoid', padding = 'same'))
    return gen

In [4]:
# Method to combine discriminator and generator models
def DCGAN(gm, dm):
    dm.trainable = False
    model = Sequential()
    model.add(gm) #adding generator model
    model.add(dm) #adding descriminator model
    o = Adam(lr = 0.0002, beta_1 = 0.5)
    model.compile(loss = 'binary_crossentropy', optimizer = o)
    return model

In [5]:
# To Load Dataset
def load_real():
    (X_train, _), (_, _) = load_data()
    X = expand_dims(X_train, axis = -1) # adding dimension to make image 3 dimensional
    X = X.astype('float32') # converting pixel values from unsigned integers to floating numbers
    X = X / 255.0 # normalizing pixel values
    return X

In [6]:
# To select training batches for each epoch
def gen_real(df, n_samples):
    n = randint(0, df.shape[0], n_samples)
    X = df[n]
    y = ones((n_samples, 1))
    return X, y

In [7]:
# To generate latent points to be converted to handwritten digits
def gen_latent(latent_dim, n_samples):
    xin = randn(latent_dim * n_samples)
    xin = xin.reshape(n_samples, latent_dim)
    return xin

In [8]:
# To generate fake samples
def gen_fake(gm, latent_dim, n_samples):
    xin = gen_latent(latent_dim, n_samples)
    X = gm.predict(xin)
    y = zeros((n_samples, 1))
    return X, y

In [9]:
# To plot the generated images
def genplot(examples , epoch, n = 5, save = False):
    for i in range(n * n):
        plt.subplot(n, n, i + 1)
        plt.axis('off')
        plt.imshow(examples[i, :, :, 0], cmap = 'gray_r')
    if(save == True):
        plt.savefig('generatedplot_e%03d.png'%(epoch+1))
    plt.close()

In [10]:
# To evaluate the model and save
def evaluate_gan(epoch, gm, dm, df, latent_dim, n_samples = 100):
    Xr, yr = gen_real(df, n_samples)
    _, acc_real = dm.evaluate(Xr, yr, verbose = 0)
    Xf, yf = gen_fake(gm, latent_dim, n_samples)
    _, acc_fake = dm.evaluate(Xf, yf, verbose = 0)
    print(">Accuracy real: %.0f%%, fake: %.0f%% " % (acc_real * 100, acc_fake * 100))
    genplot(Xf, epoch, save = True)
    gm.save("DCGAN.h5")

In [11]:
# To train DCGAN
def train(gm, dm, gan, df, latent_dim, n_epochs = 100, n_batch = 300):
    batch = int(df.shape[0]/n_batch)
    half_batch = int(n_batch/2)
    for i in range(n_epochs):
        for j in range(batch):
            X_real, y_real = gen_real(df, half_batch)
            X_fake, y_fake = gen_fake(gm, latent_dim, half_batch)
            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            d_loss, _ = dm.train_on_batch(X, y)
            X_gan = gen_latent(latent_dim, n_batch)
            y_gan = ones((n_batch, 1))
            g_loss = gan.train_on_batch(X_gan, y_gan)
            print('>%d, %d/%d, d = %.3f, g = %.3f'%(i+1, j+1, batch, d_loss, g_loss))
        if(i+1) % 10 == 0:
            evaluate_gan(i, gm, dm, df, latent_dim)

In [12]:
latent_dim = 100
d_model = discriminator()
g_model = generator(latent_dim)
gan_model = DCGAN(g_model, d_model)
dataset = load_real()
train(g_model, d_model, gan_model, dataset, latent_dim)

>1, 1/200, d = 0.709, g = 0.688
>1, 2/200, d = 0.707, g = 0.705
>1, 3/200, d = 0.693, g = 0.722
>1, 4/200, d = 0.687, g = 0.744
>1, 5/200, d = 0.682, g = 0.765
>1, 6/200, d = 0.675, g = 0.778
>1, 7/200, d = 0.665, g = 0.792
>1, 8/200, d = 0.658, g = 0.807
>1, 9/200, d = 0.656, g = 0.819
>1, 10/200, d = 0.654, g = 0.828
>1, 11/200, d = 0.648, g = 0.825
>1, 12/200, d = 0.653, g = 0.815
>1, 13/200, d = 0.655, g = 0.797
>1, 14/200, d = 0.659, g = 0.775
>1, 15/200, d = 0.664, g = 0.752
>1, 16/200, d = 0.662, g = 0.734
>1, 17/200, d = 0.663, g = 0.720
>1, 18/200, d = 0.658, g = 0.710
>1, 19/200, d = 0.655, g = 0.704
>1, 20/200, d = 0.651, g = 0.700
>1, 21/200, d = 0.643, g = 0.697
>1, 22/200, d = 0.638, g = 0.697
>1, 23/200, d = 0.634, g = 0.696
>1, 24/200, d = 0.618, g = 0.696
>1, 25/200, d = 0.607, g = 0.696
>1, 26/200, d = 0.601, g = 0.696
>1, 27/200, d = 0.593, g = 0.698
>1, 28/200, d = 0.582, g = 0.698
>1, 29/200, d = 0.577, g = 0.699
>1, 30/200, d = 0.564, g = 0.700
>1, 31/200, d = 0.5

>2, 47/200, d = 0.699, g = 0.800
>2, 48/200, d = 0.697, g = 0.785
>2, 49/200, d = 0.697, g = 0.793
>2, 50/200, d = 0.685, g = 0.801
>2, 51/200, d = 0.683, g = 0.799
>2, 52/200, d = 0.681, g = 0.807
>2, 53/200, d = 0.670, g = 0.804
>2, 54/200, d = 0.682, g = 0.800
>2, 55/200, d = 0.660, g = 0.782
>2, 56/200, d = 0.662, g = 0.787
>2, 57/200, d = 0.648, g = 0.786
>2, 58/200, d = 0.631, g = 0.789
>2, 59/200, d = 0.665, g = 0.784
>2, 60/200, d = 0.639, g = 0.797
>2, 61/200, d = 0.646, g = 0.781
>2, 62/200, d = 0.612, g = 0.768
>2, 63/200, d = 0.620, g = 0.781
>2, 64/200, d = 0.617, g = 0.795
>2, 65/200, d = 0.620, g = 0.786
>2, 66/200, d = 0.604, g = 0.794
>2, 67/200, d = 0.615, g = 0.775
>2, 68/200, d = 0.604, g = 0.793
>2, 69/200, d = 0.608, g = 0.792
>2, 70/200, d = 0.618, g = 0.762
>2, 71/200, d = 0.611, g = 0.770
>2, 72/200, d = 0.603, g = 0.756
>2, 73/200, d = 0.578, g = 0.763
>2, 74/200, d = 0.588, g = 0.759
>2, 75/200, d = 0.604, g = 0.751
>2, 76/200, d = 0.588, g = 0.748
>2, 77/200

>3, 93/200, d = 0.669, g = 0.740
>3, 94/200, d = 0.681, g = 0.756
>3, 95/200, d = 0.679, g = 0.762
>3, 96/200, d = 0.685, g = 0.776
>3, 97/200, d = 0.698, g = 0.763
>3, 98/200, d = 0.682, g = 0.757
>3, 99/200, d = 0.685, g = 0.776
>3, 100/200, d = 0.705, g = 0.763
>3, 101/200, d = 0.697, g = 0.751
>3, 102/200, d = 0.719, g = 0.750
>3, 103/200, d = 0.688, g = 0.730
>3, 104/200, d = 0.696, g = 0.751
>3, 105/200, d = 0.707, g = 0.737
>3, 106/200, d = 0.714, g = 0.738
>3, 107/200, d = 0.712, g = 0.739
>3, 108/200, d = 0.724, g = 0.734
>3, 109/200, d = 0.717, g = 0.714
>3, 110/200, d = 0.708, g = 0.703
>3, 111/200, d = 0.718, g = 0.705
>3, 112/200, d = 0.715, g = 0.723
>3, 113/200, d = 0.731, g = 0.705
>3, 114/200, d = 0.730, g = 0.709
>3, 115/200, d = 0.726, g = 0.704
>3, 116/200, d = 0.744, g = 0.689
>3, 117/200, d = 0.750, g = 0.690
>3, 118/200, d = 0.742, g = 0.674
>3, 119/200, d = 0.735, g = 0.675
>3, 120/200, d = 0.744, g = 0.682
>3, 121/200, d = 0.742, g = 0.667
>3, 122/200, d = 0.74

>4, 138/200, d = 0.711, g = 0.698
>4, 139/200, d = 0.695, g = 0.696
>4, 140/200, d = 0.698, g = 0.717
>4, 141/200, d = 0.704, g = 0.714
>4, 142/200, d = 0.701, g = 0.722
>4, 143/200, d = 0.717, g = 0.714
>4, 144/200, d = 0.696, g = 0.714
>4, 145/200, d = 0.708, g = 0.719
>4, 146/200, d = 0.697, g = 0.712
>4, 147/200, d = 0.704, g = 0.697
>4, 148/200, d = 0.702, g = 0.716
>4, 149/200, d = 0.706, g = 0.712
>4, 150/200, d = 0.692, g = 0.712
>4, 151/200, d = 0.703, g = 0.707
>4, 152/200, d = 0.702, g = 0.718
>4, 153/200, d = 0.697, g = 0.724
>4, 154/200, d = 0.698, g = 0.708
>4, 155/200, d = 0.689, g = 0.713
>4, 156/200, d = 0.698, g = 0.706
>4, 157/200, d = 0.699, g = 0.725
>4, 158/200, d = 0.684, g = 0.722
>4, 159/200, d = 0.699, g = 0.718
>4, 160/200, d = 0.688, g = 0.721
>4, 161/200, d = 0.686, g = 0.730
>4, 162/200, d = 0.682, g = 0.735
>4, 163/200, d = 0.688, g = 0.729
>4, 164/200, d = 0.684, g = 0.744
>4, 165/200, d = 0.680, g = 0.744
>4, 166/200, d = 0.676, g = 0.752
>4, 167/200, d

>5, 183/200, d = 0.691, g = 0.718
>5, 184/200, d = 0.688, g = 0.713
>5, 185/200, d = 0.697, g = 0.713
>5, 186/200, d = 0.685, g = 0.706
>5, 187/200, d = 0.681, g = 0.701
>5, 188/200, d = 0.681, g = 0.711
>5, 189/200, d = 0.688, g = 0.702
>5, 190/200, d = 0.689, g = 0.703
>5, 191/200, d = 0.685, g = 0.691
>5, 192/200, d = 0.690, g = 0.690
>5, 193/200, d = 0.689, g = 0.702
>5, 194/200, d = 0.682, g = 0.698
>5, 195/200, d = 0.690, g = 0.694
>5, 196/200, d = 0.681, g = 0.705
>5, 197/200, d = 0.693, g = 0.706
>5, 198/200, d = 0.674, g = 0.708
>5, 199/200, d = 0.691, g = 0.717
>5, 200/200, d = 0.692, g = 0.705
>6, 1/200, d = 0.680, g = 0.708
>6, 2/200, d = 0.692, g = 0.727
>6, 3/200, d = 0.687, g = 0.720
>6, 4/200, d = 0.694, g = 0.730
>6, 5/200, d = 0.680, g = 0.734
>6, 6/200, d = 0.686, g = 0.729
>6, 7/200, d = 0.687, g = 0.724
>6, 8/200, d = 0.688, g = 0.720
>6, 9/200, d = 0.693, g = 0.719
>6, 10/200, d = 0.698, g = 0.706
>6, 11/200, d = 0.696, g = 0.713
>6, 12/200, d = 0.697, g = 0.698
>

...

>95, 81/200, d = 0.695, g = 0.688
>95, 82/200, d = 0.689, g = 0.690
>95, 83/200, d = 0.694, g = 0.695
>95, 84/200, d = 0.695, g = 0.695
>95, 85/200, d = 0.692, g = 0.691
>95, 86/200, d = 0.695, g = 0.703
>95, 87/200, d = 0.692, g = 0.693
>95, 88/200, d = 0.685, g = 0.696
>95, 89/200, d = 0.686, g = 0.698
>95, 90/200, d = 0.696, g = 0.698
>95, 91/200, d = 0.692, g = 0.703
>95, 92/200, d = 0.685, g = 0.696
>95, 93/200, d = 0.689, g = 0.700
>95, 94/200, d = 0.689, g = 0.696
>95, 95/200, d = 0.698, g = 0.702
>95, 96/200, d = 0.694, g = 0.701
>95, 97/200, d = 0.690, g = 0.699
>95, 98/200, d = 0.695, g = 0.689
>95, 99/200, d = 0.691, g = 0.687
>95, 100/200, d = 0.689, g = 0.700
>95, 101/200, d = 0.691, g = 0.710
>95, 102/200, d = 0.686, g = 0.715
>95, 103/200, d = 0.685, g = 0.687
>95, 104/200, d = 0.691, g = 0.686
>95, 105/200, d = 0.696, g = 0.691
>95, 106/200, d = 0.692, g = 0.713
>95, 107/200, d = 0.696, g = 0.706
>95, 108/200, d = 0.691, g = 0.700
>95, 109/200, d = 0.690, g = 0.718
>95,

>96, 119/200, d = 0.689, g = 0.708
>96, 120/200, d = 0.688, g = 0.699
>96, 121/200, d = 0.695, g = 0.691
>96, 122/200, d = 0.689, g = 0.697
>96, 123/200, d = 0.687, g = 0.694
>96, 124/200, d = 0.688, g = 0.700
>96, 125/200, d = 0.697, g = 0.706
>96, 126/200, d = 0.690, g = 0.703
>96, 127/200, d = 0.692, g = 0.691
>96, 128/200, d = 0.687, g = 0.703
>96, 129/200, d = 0.700, g = 0.705
>96, 130/200, d = 0.696, g = 0.712
>96, 131/200, d = 0.684, g = 0.706
>96, 132/200, d = 0.687, g = 0.689
>96, 133/200, d = 0.689, g = 0.698
>96, 134/200, d = 0.693, g = 0.701
>96, 135/200, d = 0.682, g = 0.691
>96, 136/200, d = 0.686, g = 0.692
>96, 137/200, d = 0.698, g = 0.695
>96, 138/200, d = 0.693, g = 0.693
>96, 139/200, d = 0.698, g = 0.712
>96, 140/200, d = 0.693, g = 0.704
>96, 141/200, d = 0.696, g = 0.693
>96, 142/200, d = 0.695, g = 0.685
>96, 143/200, d = 0.695, g = 0.690
>96, 144/200, d = 0.688, g = 0.705
>96, 145/200, d = 0.688, g = 0.705
>96, 146/200, d = 0.690, g = 0.709
>96, 147/200, d = 0.

>97, 157/200, d = 0.692, g = 0.692
>97, 158/200, d = 0.687, g = 0.706
>97, 159/200, d = 0.692, g = 0.712
>97, 160/200, d = 0.686, g = 0.722
>97, 161/200, d = 0.688, g = 0.705
>97, 162/200, d = 0.689, g = 0.695
>97, 163/200, d = 0.691, g = 0.701
>97, 164/200, d = 0.691, g = 0.697
>97, 165/200, d = 0.685, g = 0.695
>97, 166/200, d = 0.694, g = 0.697
>97, 167/200, d = 0.687, g = 0.710
>97, 168/200, d = 0.694, g = 0.694
>97, 169/200, d = 0.693, g = 0.699
>97, 170/200, d = 0.688, g = 0.693
>97, 171/200, d = 0.683, g = 0.712
>97, 172/200, d = 0.695, g = 0.714
>97, 173/200, d = 0.696, g = 0.695
>97, 174/200, d = 0.693, g = 0.685
>97, 175/200, d = 0.697, g = 0.702
>97, 176/200, d = 0.695, g = 0.731
>97, 177/200, d = 0.689, g = 0.727
>97, 178/200, d = 0.695, g = 0.701
>97, 179/200, d = 0.688, g = 0.689
>97, 180/200, d = 0.687, g = 0.675
>97, 181/200, d = 0.690, g = 0.689
>97, 182/200, d = 0.689, g = 0.716
>97, 183/200, d = 0.694, g = 0.732
>97, 184/200, d = 0.686, g = 0.730
>97, 185/200, d = 0.

>98, 195/200, d = 0.691, g = 0.707
>98, 196/200, d = 0.694, g = 0.700
>98, 197/200, d = 0.690, g = 0.698
>98, 198/200, d = 0.688, g = 0.691
>98, 199/200, d = 0.694, g = 0.694
>98, 200/200, d = 0.694, g = 0.709
>99, 1/200, d = 0.696, g = 0.699
>99, 2/200, d = 0.689, g = 0.703
>99, 3/200, d = 0.692, g = 0.692
>99, 4/200, d = 0.691, g = 0.684
>99, 5/200, d = 0.685, g = 0.688
>99, 6/200, d = 0.693, g = 0.700
>99, 7/200, d = 0.697, g = 0.731
>99, 8/200, d = 0.692, g = 0.721
>99, 9/200, d = 0.686, g = 0.697
>99, 10/200, d = 0.696, g = 0.684
>99, 11/200, d = 0.685, g = 0.693
>99, 12/200, d = 0.692, g = 0.714
>99, 13/200, d = 0.697, g = 0.695
>99, 14/200, d = 0.694, g = 0.686
>99, 15/200, d = 0.693, g = 0.683
>99, 16/200, d = 0.688, g = 0.709
>99, 17/200, d = 0.692, g = 0.711
>99, 18/200, d = 0.691, g = 0.703
>99, 19/200, d = 0.696, g = 0.699
>99, 20/200, d = 0.690, g = 0.687
>99, 21/200, d = 0.688, g = 0.698
>99, 22/200, d = 0.692, g = 0.702
>99, 23/200, d = 0.689, g = 0.704
>99, 24/200, d = 

>100, 33/200, d = 0.693, g = 0.699
>100, 34/200, d = 0.694, g = 0.693
>100, 35/200, d = 0.699, g = 0.697
>100, 36/200, d = 0.698, g = 0.707
>100, 37/200, d = 0.691, g = 0.697
>100, 38/200, d = 0.690, g = 0.696
>100, 39/200, d = 0.694, g = 0.687
>100, 40/200, d = 0.684, g = 0.685
>100, 41/200, d = 0.686, g = 0.702
>100, 42/200, d = 0.689, g = 0.718
>100, 43/200, d = 0.694, g = 0.713
>100, 44/200, d = 0.696, g = 0.709
>100, 45/200, d = 0.687, g = 0.693
>100, 46/200, d = 0.691, g = 0.686
>100, 47/200, d = 0.695, g = 0.680
>100, 48/200, d = 0.695, g = 0.701
>100, 49/200, d = 0.686, g = 0.720
>100, 50/200, d = 0.696, g = 0.700
>100, 51/200, d = 0.692, g = 0.702
>100, 52/200, d = 0.695, g = 0.699
>100, 53/200, d = 0.693, g = 0.700
>100, 54/200, d = 0.692, g = 0.685
>100, 55/200, d = 0.687, g = 0.697
>100, 56/200, d = 0.692, g = 0.712
>100, 57/200, d = 0.692, g = 0.704
>100, 58/200, d = 0.689, g = 0.696
>100, 59/200, d = 0.684, g = 0.691
>100, 60/200, d = 0.693, g = 0.691
>100, 61/200, d = 0.